<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# 聊天机器人（Chatbot）

In [ ]:
from videos.walkthroughs import walkthrough_35 as walkthrough

In [ ]:
walkthrough()

在这个 notebook 中，您将学习如何存储对话历史，从而在基于 LLM 的链中启用聊天机器人功能。

---

## 目标

当您完成这个 notebook 时，您将会：

- 理解创建能保留对话历史的聊天机器人应用所需的核心原理和技术。
- 创建易于使用的聊天机器人，能够扮演多种不同的角色。
- 与一个简单的聊天机器人应用界面进行交互。

---

## 导入

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

---

## 创建模型实例

In [ ]:
base_url = 'http://llama:8000/v1'
model = 'meta/llama-3.1-8b-instruct'
llm = ChatNVIDIA(base_url=base_url, model=model, temperature=0)

---

## 占位符消息

在启用对话历史和聊天机器人功能之前，我们需要介绍一种尚未讲过的新类型消息，**占位符（placeholder）**消息。

简单来说，占位符消息用于在提示词模板中占据其它消息列表的位置。

In [ ]:
template_with_placeholder = ChatPromptTemplate.from_messages([
    ('placeholder', '{messages}'),
    ('human', '{prompt}')
])

In [ ]:
messages = [
    ('human', 'The sun came up today.'),
    ('ai', 'That is wonderful!'),
    ('human', 'The sun went down today.'),
    ('ai', 'That is also wonderful!.')
]

In [ ]:
prompt = 'What happened today?'

在调用（或流式处理或批处理）包含占位符消息的提示模板或链时，我们提供一个模板所需的值。而有占位符消息的话，则提供一组其它消息，而非一个字符串。

这里我们调用 `template_with_placeholder`，传入 `messages` 列表以满足模板的 `messages` 参数，并传入 `prompt` 字符串以满足 `prompt` 参数。

In [ ]:
template_with_placeholder.invoke({'messages': messages, 'prompt': prompt})

正如您所看到的，LangChain 扩展了我们提供的消息列表，形成了在调用提示模板时提供的单个消息列表。

我们可以像使用其它任何模板一样在链中使用这个提示模板。

In [ ]:
chain = template_with_placeholder | llm | StrOutputParser()

In [ ]:
chain.invoke({'messages': messages, 'prompt': prompt})

---

## 基本对话历史

我们可以用消息占位符轻松地构建一个基本的对话历史机制。首先，创建一个利用占位符的提示模板，并在一个简单的链中使用它。

In [ ]:
chat_conversation_template = ChatPromptTemplate.from_messages([
    ('placeholder', '{chat_conversation}')
])

In [ ]:
chat_chain = chat_conversation_template | llm | StrOutputParser()

接下来，创建一个列表储存对话，随着时间的推移，它会不断添加内容。

In [ ]:
chat_conversation = []

首先将添加第一个 `user` 消息到 `chat_conversation` 列表中。

In [ ]:
chat_conversation.append(('user', 'Hello, my name is Michael.'))

测试一下，现在可以用当前的 `chat_conversation` 列表调用我们的 `chat_chain`。

In [ ]:
chat_chain.invoke({'chat_conversation': chat_conversation})

看起来 LLM 能够很好地回应。不过，由于我们想要保持对话历史，再调用一次链，这次将响应作为 `ai` 消息添加到 `chat_conversation` 列表中。

In [ ]:
response = chat_chain.invoke({'chat_conversation': chat_conversation})
chat_conversation.append(('ai', response))

查看 `chat_conversation`，我们看到它现在包含了迄今为止的消息列表。

In [ ]:
chat_conversation

让我们用一条新消息重复这个过程，传入一个依赖于之前对话历史的提示。

In [ ]:
chat_conversation.append(('user', 'Do you remember what my name is?'))

In [ ]:
response = chat_chain.invoke({'chat_conversation': chat_conversation})
chat_conversation.append(('ai', response))
chat_conversation

正如您所看到的，通过将用户提示和 AI 响应分别作为 `user` 和 `ai` 消息附加到 `chat_conversation` 中，然后用整个更新后的对话调用包含占位符的 `chat_chain`，就能与 LLM 进行保留之前对话细节的对话。

基本上，所有能保留对话历史的聊天机器人功能，都是利用这种在新用户消息之前传递对话历史的方式实现的。

---

## 聊天机器人类（Chatbot Class）

我们可以将上面实现的功能封装到一个类中，这样与支持对话历史的 LLM 交互就会简单很多。请仔细阅读以下 `Chatbot` 类的定义，包括注释。

In [ ]:
class Chatbot:
    def __init__(self, llm):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('placeholder', '{chat_conversation}')
        ])

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))
        
        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

让我们实例化一个聊天机器人实例。

In [ ]:
chatbot = Chatbot(llm)

现在可以调用 `chat` 方法。

In [ ]:
print(chatbot.chat('Hi, my name is Michael.'))

In [ ]:
print(chatbot.chat('I just want to be reminded of my name please.'))

In [ ]:
print(chatbot.chat("Tell me something interesting I probably don't know about pi."))

In [ ]:
print(chatbot.chat("That's really cool! Give me another.")) # Note we are not being specific about what "another" refers to...the LLM needs to have previous messages to understand our intent.

---

## 更高级的聊天机器人

管理对话历史和创建聊天机器人都是相当广泛的话题，还有很多更高级的技术超出了本次课程的范围。不过，我们还是想为您提供一些额外的参考资料，以便进一步研究这个话题。

- [基于会话的对话历史修剪](https://python.langchain.com/docs/how_to/chatbots_memory/): LangChain 提供了一种提供历史管理能力的链封装方式，在需要管理多个会话时尤其有帮助。这一资料介绍了使用 LangChain 工具管理基于会话的对话历史，并涵盖了一些通过消息修剪和摘要来管理对话历史长度的技术，这是一个重要的话题，因为聊天对话可能会变得很长，甚至大到无法继续传给 LLM。
- [对话式 RAG](https://python.langchain.com/docs/tutorials/qa_chat_history/): 检索增强生成（Retrieval Augmented Generation，简称 RAG）（更多信息见[这个深度学习培训中心（DLI）的自学课程](https://learn.nvidia.com/courses/course-detail?course_id=course-v1:DLI+S-FX-15+V1-ZH)）是一种让 LLM 可以实时从外部数据源获取上下文，以生成响应的技术。这一资料讨论了如何在保留对话历史的聊天机器人中使用 RAG。

---

## 练习：基于角色的聊天机器人

在这个练习中，您将利用系统消息，使聊天机器人实例能够担任特定角色。

下面是 `ChatbotWithRole` 的类定义，现在与上面的 `Chatbot` 类定义完全相同，只是多了一个 `system_message` 参数（默认为空字符串），在实例化 `ChatbotWithRole` 实例时可以用它。

根据需要编辑类定义，以便您可以提供一个系统消息，为您的聊天机器人创建一个特定角色。完成后，您应该能够使用如下的系统消息为您的聊天机器人创建一个总体角色。

如果您卡住了，欢迎查看下面的*参考答案*.

In [ ]:
brief_chatbot_system_message = "You always answer as briefly and concisely as possible."

curious_chatbot_system_message = """\
You are incredibly curious, and often respond with reflections and followup questions that lean the conversation in the direction of playfully \
understanding more about the subject matters of the conversation."""

increased_vocabulary_system_message = """\
You always respond using challenging and often under-utilized vocabulary words, even when your response could be made more simply."""

### 您的代码

更新以下类定义，以便传入的 `system_message` 能有效地被聊天机器人实例使用。

In [ ]:
class ChatbotWithRole:
    def __init__(self, llm, system_message=''):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('placeholder', '{chat_conversation}')
        ])

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))
        
        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

### 试一个带角色的聊天机器人

在成功实现 `ChatbotWithRole` 后，尝试用您选择的系统消息创建一个实例并与之交互。

### 参考答案

这里，我们向 `chat_conversation_template` 添加了一个额外的系统消息，该消息使用了传入的 `system_message`。

In [ ]:
class ChatbotWithRole:
    def __init__(self, llm, system_message=''):
        # This is the same prompt template we used earlier, which a placeholder message for storing conversation history.
        chat_conversation_template = ChatPromptTemplate.from_messages([
            ('system', system_message),
            ('placeholder', '{chat_conversation}')
        ])

        # This is the same chain we created above, added to `self` for use by the `chat` method below.
        self.chat_chain = chat_conversation_template | llm | StrOutputParser()

        # Here we instantiate an empty list that will be added to over time.
        self.chat_conversation = []

    # `chat` expects a simple string prompt.
    def chat(self, prompt):
        # Append the prompt as a user message to chat conversation.
        self.chat_conversation.append(('user', prompt))
        
        response = self.chat_chain.invoke({'chat_conversation': self.chat_conversation})
        # Append the chain response as an `ai` message to chat conversation.
        self.chat_conversation.append(('ai', response))
        # Return the chain response to the user for viewing.
        return response

    # Clear conversation history.
    def clear(self):
        self.chat_conversation = []

试一个上面定义的系统消息。

In [ ]:
brief_chatbot = ChatbotWithRole(llm, system_message=brief_chatbot_system_message)
curious_chatbot = ChatbotWithRole(llm, system_message=curious_chatbot_system_message)
increased_vocabulary_chatbot = ChatbotWithRole(llm, system_message=increased_vocabulary_system_message)

In [ ]:
print(brief_chatbot.chat("What would you consider a good morning routine?"))

In [ ]:
print(curious_chatbot.chat("What would you consider a good morning routine?"))

In [ ]:
print(increased_vocabulary_chatbot.chat("What would you consider a good morning routine?"))

---

## Gradio

_“Gradio 是展示您的机器学习模型最快的方法，它提供了一个友好的 web 界面，任何人都可以在任何地方使用！”_

如果您正在构建聊天机器人，尤其是为创建原型或供个人使用的聊天机器人，可以试试 [Gradio](https://www.gradio.app/)，它可以在 Jupyter 环境中简单地创建出一个聊天界面。

将一个 `chatbot` 实例（可以通过 `Chatbot` 类或 `ChatbotWithRole` 类创建）传入以下 `create_chatbot_interface` 函数，就可以开始对话了。如果您感兴趣，可以看看 [chat_helpers/gradio_interface.py](chat_helpers/gradio_interface.py) 的源代码。

In [ ]:
from chat_helpers.gradio_interface import create_chatbot_interface

In [ ]:
app = create_chatbot_interface(curious_chatbot)
app.launch(share=True)

---

## 总结

在这个 notebook 中，您学习了如何利用一种新的消息类型，即占位符消息，来创建能够保留对话历史的聊天机器人。

这是本节的最后一个 notebook，旨在使用聊天消息类型来提升 LLM 应用的效果。除了管理对话历史外，您还学习了许多技术，包括少样本提示、使用系统消息及执行思维链提示。

在接下来的部分中，您将专注于使用多种提示工程技术，使您的基于 LLM 的应用能生成结构化数据，这是一种强大的能力，可以让您的 LLM 应用更及时地与下游代码交互，并为借助 LLM 标记（tag）和分析大量文本数据打开了无限可能。